# <center> Logistic Regression </center>
<center> Corinne Jones, TA </center>
<center> DATA 558, Spring 2020 </center>

In this lab we'll see how to use scikit-learn's logistic regression function. After this lab, you should know
how to fit a logistic regression model with scikit-learn.

## 1 Logistic Regression 
Recall that in logistic regression we have a binary response variable $y$ and use the model
$$ P(y=1|X; \beta) = \frac{\exp{(\beta_0+\beta_1X_1 + \cdots + \beta_dX_d)}}{1+\exp{(\beta_0+\beta_1X_1 + \cdots + \beta_dX_d)}}.$$
By transforming the linear combination of predictors, $\beta_0 + \beta_1X_1 + \cdots + \beta_dX_d$, in the above equation we ensure that $P(y=1|X;\beta)$ (the estimated probability that the response is equal to 1, conditional on the predictors $X=(X_1,\dots,X_d)$) is always between 0 and 1. 
We can also rearrange the above equation to get
$$ \log\left(\frac{P(y=1|X; \beta)}{1-P(y=1|X; \beta)}\right) = \beta_0 + \beta_1X_1 + \cdots + \beta_dX_d.$$

During the lecture you saw that when the labels are in the set $\{-1, +1\}$ (as opposed to, e.g.,  $\{0, 1\}$), maximizing the log-likelihood entails solving the problem 
$$\min_{\beta_0\in\mathbb{R}, \beta \in \mathbb{R}^d} \: \frac{1}{n} \sum_{i=1}^n \log\left(1+ \exp(-y_i(\beta_0+x_i^T \beta))\right).$$
where $\beta=[\beta_1,\dots, \beta_d]^T$.

After fitting the model we typically label a new input $x$ with 1 if our estimated probability of it being 1, $P(y=1|x; \beta^\star)$, is larger than the estimated probability of it being -1, $P(y=-1|x;\beta^\star)$ (or 0, depending on the convention). 

**Exercise 1.** Denote by $\beta_0^\star$, $\beta^\star$ the $\beta_0,\beta$ that maximize the log-likelihood. Show that $P(y=1|x;\beta^\star)>P(y=-1|x;\beta^\star)$ if and only if $\beta_0^\star + (\beta^\star)^T x > 0$.

### 1.1 Log sum exp trick
In practice, $\exp(x_i^T\beta)$ could be very large or very close to zero, resulting in overflow or underflow. Suppose we want to compute $\log(1+\exp(1000))$:

In [ ]:
import numpy as np
np.log(1+np.exp(1000))

It appears as though we get an overflow error! But it should be finite, since $1+\exp(1000)\approx \exp(1000)$ and hence $\log(1+\exp(1000)) \approx \log(\exp(1000)) = 1000$. To circumvent this problem, you can use a trick: the fact that
$$ \log(1+\exp(x)) = a + \log(\exp(-a) + \exp(x-a))$$
for any $a$.

**Exercise 2.** Show that the above identity is true.

**Exercise 3.** Use the above identity to compute $\log(1+\exp(1000))$.

In [7]:
a = 800
x = 1000
a + np.log(np.exp(-a)+np.exp(x-a))

1000.0

You should get something close to 1000.

### 1.2 Loading the data
Next we'll use logistic regression to distinguish Hornbills and Toucans. The features that we'll use are derived from images. You can download the data from Canvas.

**To do:** Change the directory below (if necessary) to load in the data for this lab.

In [10]:
import numpy as np
import os

data_dir = 'lab3_data'

x_train = np.load(os.path.join(data_dir, 'train_features.npy'))
y_train = np.load(os.path.join(data_dir, 'train_labels.npy'))
x_test = np.load(os.path.join(data_dir, 'test_features.npy'))
y_test = np.load(os.path.join(data_dir, 'test_labels.npy'))

"Hornbill" has label 0 and "Toucan" has label 1. There are 500 images in the training set and 100 images in the test set. There is one *row* corresponding to each image.

In [11]:
print('Number of images:', x_train.shape[0])
print('Dimension of features:', x_train.shape[1])

Number of images: 1000
Dimension of features: 4096


### 1.3 Classification using Scikit-Learn
Here we're going to use logistic regression from Scikit-learn to classify the images. 

**Exercise 4.** Standardize the data. You may use Scikit-learn's `StandardScaler` for this. Use the same names for the normalized data (`x_train` and `x_test`).  
Reference: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)


As with linear regression, when using logistic regression it is beneficial to add a penalty term on the norm of the weights. With this penalty, we then optimize the expression
$$\min_{\beta \in \mathbb{R}^d}\; \frac{1}{n} \sum_{i=1}^n \log\left(1+ \exp(-y_i(\beta_0+x_i^T \beta))\right) + \lambda \|\beta\|^2_2.$$
(where the labels are $\pm 1$). We choose $\lambda$ via cross-validation.

**Exercise 5.** Use `LogisticRegressionCV` to fit the model to the training data. For today use the default parameter values. Then compute the accuracy on the test set. Use `classifier` as the name of your instantiation of `LogisticRegressionCV`.
Reference: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html  

We're able to get 88% accuracy! Let's get an idea of what images it classified correctly and incorrectly.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

def display_ranked_image_list(names, image_dir, scores, num_images=10, cutoff=0.5, true_labels=None, display_mistakes=False):
    """
    Display a (subset of a) ranked list of images. By default, this function displays
    10 images from the list of image names ("names") sorted by decreasing scores ("scores").
    :param names: List of image names
    :param scores: Scores for each image (from some classifier)
    :param num_images: Number of images to display
    :param true_labels: The true labels of each image
    :param display_mistakes: Whether to only display the top images on which the classifier made mistakes
    """
    ncol = 6
    
    idxs = np.argsort(scores)
    if not display_mistakes:
        idxs = idxs[-num_images:]
    else:
        mistakes = np.where(true_labels != 1)[0]
        idxs = [i for i in idxs if (i in mistakes and scores[i] > cutoff)][-num_images:]
    
    num_images = len(idxs)
    nrow = int(num_images/6) + 1 if num_images % 6 != 0 else int(num_images/6)
    fig = plt.figure()
    fig.set_figwidth(15)
    fig.set_figheight(5*nrow/2)
    for i in range(1, num_images+1):
        idx = idxs[i-1]
        a = fig.add_subplot(nrow, 6, num_images-i+1)
        img = mpimg.imread(os.path.join(image_dir, names[idx]))
        imgplot = plt.imshow(img)
        a.set_title(scores[idx])
        plt.axis('off')
    plt.show()

In [ ]:
# Get names of files in validation set
image_names = sorted(os.listdir(os.path.join(data_dir, 'images', 'test')))
image_dir = os.path.join(data_dir, 'images', 'test')

# Generate estimated values for test observations using the logistic regression classifier
test_probs = classifier.predict_proba(x_test)

# See the images it was most confident were toucans that were in fact toucans
print('Images it was most confident were toucans that were in fact toucans:')
display_ranked_image_list(image_names, image_dir, test_probs[:, 1])

#  See the images it was most confident about being toucans that were not toucans
print('Images it was most confident about being toucans that were not toucans:')
display_ranked_image_list(image_names, image_dir, test_probs[:, 1], true_labels=y_test, display_mistakes=True)

# See the images it was most confident were hornbills
print('Images it was most confident were hornbills that were in fact hornbills:')
display_ranked_image_list(image_names, image_dir, 1-test_probs[:, 1])

#  See the images it was most confident about being hornbills that were not hornbills
print('Images it was most confident about being hornbills that were not hornbills:')
display_ranked_image_list(image_names, image_dir, 1-test_probs[:, 1], true_labels=(y_test-1)*-1, display_mistakes=True)


Up until now we've only discussed accuracy as a way of measuring the quality of a classifier. However, it turns out that there are also other ways to do so. We'll see these in a future lab.